# Ejercicio de obtener las recetas de ensaladas de recetasgratis.net

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"
response = requests.get(url)
bool(response)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:

recetas = soup.find("div", attrs = {'class':'clear padding-left-1'}).find_all("div", class_="resultado link")

len(recetas)

50

In [7]:
recetas[0].find("a").text

'Receta de Ensalada coleslaw'

In [8]:
titulos = [i.find("a").text for i in recetas]
titulos[:4]

['Receta de Ensalada coleslaw',
 'Receta de Ensalada de quinoa y verduras',
 'Ensaladas especiales para fiestas',
 'Receta de Ensalada navideña peruana']

In [9]:
links = [i.find("a")["href"] for i in recetas]
links[:4]

['https://www.recetasgratis.net/receta-de-ensalada-coleslaw-77205.html',
 'https://www.recetasgratis.net/receta-de-ensalada-de-quinoa-y-verduras-77158.html',
 'https://www.recetasgratis.net/articulo-ensaladas-especiales-para-fiestas-77062.html',
 'https://www.recetasgratis.net/receta-de-ensalada-navidena-peruana-76980.html']

### Hasta este punto se obtuvieron los links y titulos de las 50 recetas de la primera página, ahora se obtendran todos los titulos y links de los 850 recetas

In [10]:
len(links)

50

In [11]:
np.ceil(701/50)

15.0

In [14]:
links_all = []
titulos_all = []
for i in range(1,15+1):
    print(i)
    url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_{}.html".format(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    recetas = soup.find("div", attrs = {'class':'clear padding-left-1'}).find_all("div", class_="resultado link")
    titulos = [i.find("a",{"class":"titulo titulo--resultado"}).text for i in recetas]
    links = [i.find("a",{"class":"titulo titulo--resultado"})["href"] for i in recetas]
    links_all.extend(links)
    titulos_all.extend(titulos)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [15]:
len(links_all)

701

In [16]:
len(titulos_all)

701

### En la siguiente sección se obtene la info de una sola receta que se quiere

In [17]:
url = links[0]
response = requests.get(url)
bool(response)
soup = BeautifulSoup(response.text, "html.parser")

In [18]:
url

'https://www.recetasgratis.net/receta-de-ensalada-azteca-6623.html'

In [19]:
recipe_info  = soup.find("div", class_="recipe-info")
ingredientes = ", ".join([i.text.strip() for i in recipe_info.find_all("li", class_="ingrediente")])
if soup.find(class_ = "properties") is not None:
    propiedades = [i.text for i in soup.find(class_ = "properties").find_all("span")]
else:
    propiedades = None
ingredientes, propiedades

('1 Lechuga Romana, Queso ranchero, Chile negro frito, 1 Aguacate, 1 Tortilla frita en juliana., Para la Vinagreta, 1 diente de ajo picado muy fino, 1 cucharadita de cebolla morada muy finita, 1 pizca de chile de árbol molido, Zumo de un limón, 1 shot de vainilla, Aceite de olivo, el necesario, Sal, pimienta al gusto.',
 None)

### Se crea una función para automatizar

In [20]:
def obtener_info_receta(link_receta):
    response = requests.get(link_receta)
    soup = BeautifulSoup(response.text, "html.parser")
    recipe_info  = soup.find("div", class_="recipe-info")
    ingredientes = ", ".join([i.text.strip() for i in recipe_info.find_all("li", class_="ingrediente")])
    if soup.find(class_ = "properties") is not None:
        propiedades = [i.text for i in soup.find(class_ = "properties").find_all("span")]
    else:
        propiedades = None
    return [ingredientes, propiedades]

# Se recorren todos los links con la función anterior 

In [21]:
# Este for tarda unos 5 minutos
# Algunas recetas no tienen el formato adecuado, pasamos de ellas
info_receta=[]
for link in links_all:
    try:
        info_receta.append(obtener_info_receta(link))
    except:
        print(link)

https://www.recetasgratis.net/articulo-ensaladas-especiales-para-fiestas-77062.html
https://www.recetasgratis.net/articulo-recetas-de-ensaladas-proteicas-75344.html
https://www.recetasgratis.net/articulo-ensaladas-para-navidad-originales-74827.html
https://www.recetasgratis.net/video-receta-de-ensalada-cesar-para-tupper-58655.html


In [14]:
df = pd.DataFrame(info_receta, columns =["Ingredientes","propiedades"])


In [15]:
df

,Ingredientes,propiedades
0,"4 papas blancas o rosadas, 2 zanahorias, 200 g...","[10 comensales, 30m, Dificultad baja]"
1,"250 gramos de pasta corta (canuto, tornillitos...","[5 comensales, 30m, Dificultad baja]"
2,"4 patatas, 2 zanahorias, 100 gramos de guisant...","[4 comensales, 45m, Dificultad media]"
3,"500 gramos de garbanzos cocidos, 1 cebolleta, ...","[4 comensales, 15m, Dificultad baja]"
4,"1 lata de de 800 gr. de tomates enteros, 2 lat...","[4 comensales, 15m, Dificultad baja]"
...,...,...
676,"1 paquete de Surimi, 1 lata de Media crema, 1 ...","[2 comensales, 15m, Entrante, Dificultad baja]"
677,"100 gramos de Palitos de cangrejo o Surimir, 1...","[6 comensales, 10m, Entrante, Dificultad baja]"
678,"4 hojas de Lechuga, 1 unidad de Manzana roja, ...","[2 comensales, 10m, Entrante, Dificultad baja]"
679,"1 lata de Garbanzos, 1 lata de Habichuelas Col...","[4 comensales, 30m, Entrante, Dificultad baja]"
